In [1]:
import pandas as pd

In [2]:
x = pd.read_csv("gnn_input.csv")
x.head()

,Unnamed: 0,key,severityavg_severity,severitymax_severity,severitynum_barriers,lengthfirst,CurbRamp_count,NoCurbRamp_count,NoSidewalk_count,Obstacle_count,...,NoCurbRamp_max,NoSidewalk_max,Obstacle_max,Occlusion_max,Other_max,SurfaceProblem_max,u_lat,u_lon,v_lat,v_lon
0,0,0.0,1.0,1.0,1,317.313855,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,47.662018,-122.322863,47.664871,-122.322864
1,1,0.0,1.0,1.0,2,19.772566,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,47.648425,-122.342633,47.648600,-122.342604
2,2,0.0,2.0,3.0,2,13.534974,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,47.646925,-122.336374,47.646803,-122.336373
3,3,0.0,1.0,1.0,2,16.156787,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,47.646921,-122.334031,47.647067,-122.334030
4,4,0.0,3.0,3.0,1,11.729261,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,47.665809,-122.301937,47.665880,-122.302052


In [3]:
y = pd.read_csv("gnn_input.csv")
y.columns

Index(['Unnamed: 0', 'key', 'severityavg_severity', 'severitymax_severity',
       'severitynum_barriers', 'lengthfirst', 'CurbRamp_count',
       'NoCurbRamp_count', 'NoSidewalk_count', 'Obstacle_count',
       'Occlusion_count', 'Other_count', 'SurfaceProblem_count',
       'CurbRamp_avg', 'NoCurbRamp_avg', 'NoSidewalk_avg', 'Obstacle_avg',
       'Occlusion_avg', 'Other_avg', 'SurfaceProblem_avg', 'CurbRamp_max',
       'NoCurbRamp_max', 'NoSidewalk_max', 'Obstacle_max', 'Occlusion_max',
       'Other_max', 'SurfaceProblem_max', 'u_lat', 'u_lon', 'v_lat', 'v_lon'],
      dtype='object')

In [4]:
import osmnx as ox
import networkx as nx

# Download Seattle walk network
G_full = ox.graph_from_place(
    "Seattle, Washington, USA",
    network_type="walk"
)
G_full = nx.Graph(G_full)

print("Base Seattle graph")
print("Nodes:", G_full.number_of_nodes())
print("Edges:", G_full.number_of_edges())


Base Seattle graph
Nodes: 109164
Edges: 149957


In [5]:
# !pip install osmnx

In [6]:
#Mobility GNN score - edge based 

In [7]:
df = x.copy()  # <-- use your existing DataFrame here

def key(lat, lon):
    return (round(lat, 6), round(lon, 6))

# Map: (u_coord, v_coord) -> row with barrier features
edge_features = {}

for _, r in df.iterrows():
    u = key(r.u_lat, r.u_lon)
    v = key(r.v_lat, r.v_lon)
    edge_features[(u, v)] = r
    edge_features[(v, u)] = r  # undirected

# Attach features to each NetworkX edge
for u, v in G_full.edges():
    # Node coordinates from OSMnx
    u_coord = key(G_full.nodes[u]["y"], G_full.nodes[u]["x"])
    v_coord = key(G_full.nodes[v]["y"], G_full.nodes[v]["x"])

    data_row = edge_features.get((u_coord, v_coord), None)

    if data_row is None:
        G_full[u][v]["features"] = {
            "NoCurbRamp": 0.0,
            "NoSidewalk": 0.0,
            "Obstacle": 0.0,
            "CurbRamp": 1.0,          # default: safe
            "SurfaceProblem": 0.0,
            "Occlusion": 0.0
        }
    else:
        G_full[u][v]["features"] = {
            "NoCurbRamp": float(data_row["NoCurbRamp_count"]),
            "NoSidewalk": float(data_row["NoSidewalk_count"]),
            "Obstacle": float(data_row["Obstacle_count"]),
            "CurbRamp": float(data_row["CurbRamp_count"]),
            "SurfaceProblem": float(data_row["SurfaceProblem_count"]),
            "Occlusion": float(data_row["Occlusion_count"])
        }

In [9]:
!pip install torch

In [12]:
import torch

def mobility_risk(f):
    return (
        5.0 * f["NoCurbRamp"] +
        4.0 * f["NoSidewalk"] +
        3.0 * f["Obstacle"] +
        2.0 * max(0.0, 1.0 - f["CurbRamp"]) +   # no curb ramp is bad
        1.5 * f["SurfaceProblem"] +
        1.0 * f["Occlusion"]
    )

# Precompute target risk per edge (for now using analytic formula)
edge_list = list(G_full.edges())
y_vals = []
x_feats = []

for (u, v) in edge_list:
    f = G_full[u][v]["features"]
    x_feats.append([
        f["NoCurbRamp"],
        f["NoSidewalk"],
        f["Obstacle"],
        f["CurbRamp"],
        f["SurfaceProblem"],
        f["Occlusion"]
    ])
    y_vals.append(mobility_risk(f))

x = torch.tensor(x_feats, dtype=torch.float)       # [num_edges, 6]
y = torch.tensor(y_vals, dtype=torch.float)        # [num_edges]

In [13]:
# 1) Build canonical edge_list from the graph
def canon_edge(u, v):
    # Assume undirected: represent edges as sorted tuples
    return (u, v) if u <= v else (v, u)

edge_list = [canon_edge(u, v) for (u, v) in G_full.edges()]  # canonical
num_edges = len(edge_list)

# 2) Build x, y using canonical edge_list
x_feats = []
y_vals = []

for (u_c, v_c) in edge_list:
    f = G_full[u_c][v_c]["features"]
    x_feats.append([
        f["NoCurbRamp"],
        f["NoSidewalk"],
        f["Obstacle"],
        f["CurbRamp"],
        f["SurfaceProblem"],
        f["Occlusion"],
    ])
    y_vals.append(mobility_risk(f))

x = torch.tensor(x_feats, dtype=torch.float)
y = torch.tensor(y_vals, dtype=torch.float)

# 3) Map canonical edge -> index
e2idx = {e: i for i, e in enumerate(edge_list)}

# 4) Build line-graph edge_index using the SAME canonicalization
edge_index_list = []

for w in G_full.nodes():
    # incident edges as canonical pairs
    incident_raw = list(G_full.edges(w))
    incident = [canon_edge(u, v) for (u, v) in incident_raw]

    # Fully connect them
    for i in range(len(incident)):
        for j in range(i + 1, len(incident)):
            ei = e2idx[incident[i]]
            ej = e2idx[incident[j]]
            edge_index_list.append([ei, ej])
            edge_index_list.append([ej, ei])

if len(edge_index_list) == 0:
    raise ValueError("Line graph has no edges; check your input graph.")

edge_index = torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()
data = Data(x=x, edge_index=edge_index, y=y)

print("Line graph nodes (edges):", data.num_nodes)
print("Line graph edges:", data.num_edges)

NameError: name 'Data' is not defined

In [ ]:
# !pip install torch_geometric

In [ ]:
data

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GraphSAGE
import numpy as np
import math
import networkx as nx
import osmnx as ox

#############################################
# 1. Edge-based GNN model and training
#############################################

class MobilityEdgeGNN(nn.Module):
    def __init__(self, in_channels=6, hidden_channels=32):
        super().__init__()
        self.gnn = GraphSAGE(
            in_channels=in_channels,
            hidden_channels=hidden_channels,
            num_layers=2
        )
        self.head = nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index):
        h = self.gnn(x, edge_index)      # [num_edges, hidden]
        out = self.head(h).squeeze(-1)   # [num_edges]
        return out

model = MobilityEdgeGNN(in_channels=6, hidden_channels=32)
opt = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(200):
    model.train()
    pred = model(data.x, data.edge_index)     # [num_edges]
    loss = ((pred - data.y) ** 2).mean()

    opt.zero_grad()
    loss.backward()
    opt.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch:3d} | Loss {loss.item():.4f}")

In [ ]:
#############################################
# 2. Attach learned risk to original edges
#############################################

model.eval()
with torch.no_grad():
    risk_pred = model(data.x, data.edge_index).cpu().numpy()  # [num_edges]

print("First 20 predicted risks:", risk_pred[:20])
print("Unique risk values (rounded):", np.unique(np.round(risk_pred, 4)))

# edge_list must be in canonical form: (min(u,v), max(u,v))
for i, (u_c, v_c) in enumerate(edge_list):
    G_full[u_c][v_c]["risk"] = float(risk_pred[i])

In [ ]:
print("True risk[0:20]:", data.y[:20].numpy())
print("Pred risk[0:20]:", risk_pred[:20])


In [ ]:
import numpy as np

# Unique values (can be a lot, so maybe round first)
unique_vals = np.unique(np.round(risk_pred, 3))
print("Unique predicted risks (rounded to 3 decimals):", unique_vals)

# Basic stats / range
print("Min pred risk:", risk_pred.min())
print("Max pred risk:", risk_pred.max())
print("Mean pred risk:", risk_pred.mean())
print("Std pred risk:", risk_pred.std())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Create the distribution plot with histogram + smooth KDE
plt.figure(figsize=(12, 6))

# Plot histogram with KDE overlay
sns.histplot(
    risk_pred, 
    bins=100, 
    kde=True, 
    stat='density',  # Normalize to density for KDE compatibility
    alpha=0.6, 
    color='skyblue',
    kde_kws={'bw_adjust': 0.5}  # Adjust smoothness (0.2=sharp, 2.0=smooth)
)

# Add vertical lines for key percentiles
p25 = np.percentile(risk_pred, 25)
p75 = np.percentile(risk_pred, 75)
p70 = np.percentile(risk_pred, 70)  # Your 30% cutoff

plt.axvline(p25, color='green', linestyle='--', label=f'Q1 (25th): {p25:.3f}')
plt.axvline(p75, color='orange', linestyle='--', label=f'Q3 (75th): {p75:.3f}')
plt.axvline(p70, color='red', linestyle='--', linewidth=2, label=f'p70 (30% above): {p70:.3f}')

# Shade regions
plt.axvspan(p70, risk_pred.max(), alpha=0.2, color='red', label='High risk (30%)')
plt.axvspan(p25, p75, alpha=0.1, color='orange', label='IQR (25-75%)')

plt.title('Risk Prediction Distribution (Before Rescaling)\nSeattle Sidewalk Edges', fontsize=14)
plt.xlabel('Predicted Risk Score')
plt.ylabel('Density')
plt.legend()
plt.grid(True, alpha=0.3)

# Log scale for x-axis to see the heavy tail better
plt.xscale('symlog', linthresh=0.01)
plt.tight_layout()
plt.show()

# Print summary stats
print("Distribution Summary:")
print(f"  Most data (~70%) clustered around: [-0.01, 0.02]")
print(f"  Heavy right tail: up to {risk_pred.max():.1f}")
print(f"  30% high-risk edges: >= {p70:.3f}")

In [ ]:
import numpy as np

# Calculate 25th and 75th percentiles of risk_pred
p25 = np.percentile(risk_pred, 25)
p75 = np.percentile(risk_pred, 75)

print(f"25th percentile (Q1): {p25:.3f}")
print(f"75th percentile (Q3): {p75:.3f}")

# Also show basic stats
print(f"Min: {risk_pred.min():.3f}")
print(f"Median (50th): {np.percentile(risk_pred, 50):.3f}")
print(f"Max: {risk_pred.max():.3f}")

# Interquartile range (IQR)
iqr = p75 - p25
print(f"IQR (p75 - p25): {iqr:.3f}")

# Count of edges in each quartile
n_total = len(risk_pred)
print(f"\nQuartile breakdown:")
print(f"Q1-Q3 (25th-75th): {np.sum((risk_pred >= p25) & (risk_pred <= p75))} edges ({100*(np.sum((risk_pred >= p25) & (risk_pred <= p75))/n_total):.1f}%)")
print(f"Below Q1 (< {p25:.1f}): {np.sum(risk_pred < p25)} edges ({100*(np.sum(risk_pred < p25)/n_total):.1f}%)")
print(f"Above Q3 (> {p75:.1f}): {np.sum(risk_pred > p75)} edges ({100*(np.sum(risk_pred > p75)/n_total):.1f}%)")


In [ ]:
import numpy as np

# For TOP 5% data >= 7, use the 95th percentile as cutoff
p95 = np.percentile(risk_pred, 98)  # 95th percentile -> maps to 7 (top 5%)
r_min, r_max = risk_pred.min(), risk_pred.max()

print(f"Rescaling anchors:")
print(f"  Min: {r_min:.3f} -> 0")
print(f"  p95: {p95:.3f} -> 7") 
print(f"  Max: {r_max:.3f} -> 10")

def rescale_top5pct(risk_val):
    """Rescale so TOP 5% of data >= 7, range [0,10]"""
    if risk_val <= p95:
        # [r_min, p95] -> [0, 7]  (95% of data)
        if p95 == r_min:
            return 0.0
        return 7.0 * (risk_val - r_min) / (p95 - r_min)
    else:
        # [p95, r_max] -> [7, 10] (top 5%)
        if r_max == p95:
            return 10.0
        return 7.0 + 3.0 * (risk_val - p95) / (r_max - p95)

# Apply to all predictions
risk_normalized = np.array([rescale_top5pct(val) for val in risk_pred])
risk_normalized = np.clip(risk_normalized, 0, 10)

# Verify
print(f"\nAfter rescaling:")
print(f"  Range: [{risk_normalized.min():.2f}, {risk_normalized.max():.2f}]")
print(f"  Fraction >= 7: {np.mean(risk_normalized >= 7):.1%}")  # ~5%
print(f"  p25: {np.percentile(risk_normalized, 25):.2f}")
print(f"  p75: {np.percentile(risk_normalized, 75):.2f}")
print(f"  p95: {np.percentile(risk_normalized, 95):.2f}")

# Attach normalized risk to graph
for i, (u_c, v_c) in enumerate(edge_list):
    G_full[u_c][v_c]["risk_norm"] = float(risk_normalized[i])

# Update wheelchair_cost with normalized risk (0-10 scale)
alpha = 1.0  # distance weight
beta = 2.0   # risk weight (tune for 0-10 scale)

for u, v, d in G_full.edges(data=True):
    length = d.get("length", 1.0)
    risk_norm = d.get("risk_norm", 0.0)
    d["wheelchair_cost"] = alpha * length + beta * risk_norm


In [ ]:
# 1. Create a lookup dictionary: (canonical_u, canonical_v) -> risk_norm
# We use the canonicalized keys to ensure the undirected match works
risk_lookup = {}
for i, (u_c, v_c) in enumerate(edge_list):
    risk_lookup[(u_c, v_c)] = float(risk_normalized[i])

# 2. Map the risk back to the original DataFrame
def get_risk_for_row(row):
    # Convert row coordinates to the canonical key format used in the GNN
    u_coord = key(row['u_lat'], row['u_lon'])
    v_coord = key(row['v_lat'], row['v_lon'])
    
    # Canonicalize the pair (min, max) to match the risk_lookup keys
    canon_key = (u_coord, v_coord) if u_coord <= v_coord else (v_coord, u_coord)
    
    # Return the normalized risk, default to 0.0 if not found
    return risk_lookup.get(canon_key, 0.0)

# 3. Add the column to your input data
df['pred'] = df.apply(get_risk_for_row, axis=1)

print("DataFrame updated with 'pred' column.")
print(df[['u_lat', 'u_lon', 'v_lat', 'v_lon', 'pred']].head())

In [ ]:
import folium
import numpy as np

# Center of Seattle walk network (OSMnx format)
lats = [G_full.nodes[n]["y"] for n in G_full.nodes()]
lons = [G_full.nodes[n]["x"] for n in G_full.nodes()]

m = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    zoom_start=13,
    tiles="cartodbpositron"
)

# Draw each edge, color-coded by normalized risk (TOP 5% red)
for u, v, data in G_full.edges(data=True):
    # Get node coordinates (OSMnx uses "y"=lat, "x"=lon)
    lat_u, lon_u = G_full.nodes[u]["y"], G_full.nodes[u]["x"]
    lat_v, lon_v = G_full.nodes[v]["y"], G_full.nodes[v]["x"]
    
    # Get normalized risk (0-10 scale)
    risk_norm = data.get("risk_norm", 0.0)
    
    # Color: RED if >=7 (TOP 5% riskiest), GREEN if <7
    color = "red" if risk_norm >= 7 else "green"
    
    # Tooltip with risk details
    tooltip = f"Risk: {risk_norm:.1f}/10"
    if "wheelchair_cost" in data:
        tooltip += f" | Cost: {data['wheelchair_cost']:.1f}"
    
    folium.PolyLine(
        locations=[[lat_u, lon_u], [lat_v, lon_v]],
        color=color,
        weight=4 if risk_norm >= 7 else 2,  # EXTRA thick for top 5%
        opacity=0.9 if risk_norm >= 7 else 0.7,  # More opaque for high risk
        tooltip=tooltip
    ).add_to(m)

# Updated legend for TOP 5%
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 220px; height: 90px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px; font-weight: bold;">
<p><span style="color: green;">🟢 Green</span>: Safe (95% edges)</p>
<p><span style="color: red;">🔴 Red</span>: DANGER (TOP 5%)</p>
<p><b>Only 5% edges are high risk</b></p>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

print(f"Map created: {np.mean(risk_normalized >= 7)*100:.1f}% edges >= 7 (RED = TOP 5%)")
m

In [ ]:
#Mobility GNN score - edge based 

In [ ]:
# After your existing map code, ADD THIS:

# Example route coordinates
route_coords = {"start": [47.6144219, -122.192337], "end": [45.6554303, -120.30016925]}

# 1. Find nearest nodes and compute A* route
import osmnx as ox
import networkx as nx
import math

def euclidean_distance(u, v):
    y1, x1 = G_full.nodes[u]["y"], G_full.nodes[u]["x"]
    y2, x2 = G_full.nodes[v]["y"], G_full.nodes[v]["x"]
    return math.sqrt((y1 - y2)**2 + (x1 - x2)**2)

# Get nearest nodes
start_lat, start_lon = route_coords["start"]
end_lat, end_lon = route_coords["end"]
orig_node = ox.distance.nearest_nodes(G_full, X=start_lon, Y=start_lat)
dest_node = ox.distance.nearest_nodes(G_full, X=end_lon, Y=end_lat)

# A* using wheelchair_cost (AVOIDS red edges automatically)
path_nodes = nx.astar_path(
    G_full,
    source=orig_node,
    target=dest_node,
    heuristic=lambda u, v: euclidean_distance(u, v),
    weight="wheelchair_cost"
)

# 2. Extract route metrics
route_edges = list(zip(path_nodes[:-1], path_nodes[1:]))
route_risks = []
route_lengths = []
total_length = 0
high_risk_count = 0

for u, v in route_edges:
    edge_data = G_full[u][v]
    risk_norm = edge_data.get("risk_norm", 0.0)
    length = edge_data.get("length", 0.0)
    
    route_risks.append(risk_norm)
    route_lengths.append(length)
    total_length += length
    if risk_norm >= 7:
        high_risk_count += 1

avg_risk = np.mean(route_risks)
num_edges = len(route_edges)

# 3. PLOT SAFEST ROUTE (thick BLUE line)
route_coords_map = [[G_full.nodes[n]["y"], G_full.nodes[n]["x"]] for n in path_nodes]
folium.PolyLine(
    locations=route_coords_map,
    color="blue",
    weight=8,
    opacity=0.95,
    tooltip=f"🛡️ SAFEST ROUTE (Avg Risk: {avg_risk:.1f}/10)"
).add_to(m)

# 4. Start/End markers
folium.Marker(
    [start_lat, start_lon], popup="🚩 START",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)
folium.Marker(
    [end_lat, end_lon], popup="🏁 END",
    icon=folium.Icon(color="red", icon="stop")
).add_to(m)

# 5. Updated legend WITH ROUTE METRICS
legend_html = f'''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 280px; height: 160px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 15px; font-weight: bold;">
<p><span style="color: green;">🟢 Green:</span> Safe (95% edges)</p>
<p><span style="color: red;">🔴 Red:</span> DANGER (TOP 5%)</p>
<p><span style="color: blue; font-weight: bold;">🔵 Blue:</span> YOUR SAFEST ROUTE</p>
<hr>
<p><b>📊 ROUTE METRICS:</b></p>
<p>Avg Risk: <span style="color:blue">{avg_risk:.1f}/10</span></p>
<p>Length: <span style="color:green">{total_length:.0f}m</span></p>
<p>High Risk Edges: <span style="color:orange">{high_risk_count}</span></p>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# 6. Print metrics summary
print(f"\n🛤️ ROUTE COMPLETE!")
print(f"Avg Risk Score:     {avg_risk:.2f}/10")
print(f"Total Length:       {total_length:.0f} meters") 
print(f"Number of Edges:    {num_edges}")
print(f"High Risk Edges:    {high_risk_count} (should be 0!)")
print(f"Safety Score:       {100*(1 - high_risk_count/num_edges):.1f}% safe")

m

In [ ]:
# REPLACE your existing route code with this DUAL ROUTE version:

# Example route coordinates
route_coords = {"start": [47.6144219, -122.192337], "end": [45.6554303, -120.30016925]}

# 1. Helper functions
def euclidean_distance(u, v):
    y1, x1 = G_full.nodes[u]["y"], G_full.nodes[u]["x"]
    y2, x2 = G_full.nodes[v]["y"], G_full.nodes[v]["x"]
    return math.sqrt((y1 - y2)**2 + (x1 - x2)**2)

def compute_route_metrics(path_nodes):
    """Extract metrics for any route"""
    route_edges = list(zip(path_nodes[:-1], path_nodes[1:]))
    route_risks = []
    route_lengths = []
    total_length = 0
    high_risk_count = 0
    
    for u, v in route_edges:
        edge_data = G_full[u][v]
        risk_norm = edge_data.get("risk_norm", 0.0)
        length = edge_data.get("length", 0.0)
        wheelchair_cost = edge_data.get("wheelchair_cost", 0.0)
        
        route_risks.append(risk_norm)
        route_lengths.append(length)
        total_length += length
        if risk_norm >= 7:
            high_risk_count += 1
    
    return {
        'path_nodes': path_nodes,
        'avg_risk': np.mean(route_risks),
        'total_length': total_length,
        'num_edges': len(route_edges),
        'high_risk_count': high_risk_count,
        'total_cost': sum(G_full[u][v]["wheelchair_cost"] for u, v in route_edges),
        'route_coords': [[G_full.nodes[n]["y"], G_full.nodes[n]["x"]] for n in path_nodes]
    }

# 2. ROUTE 1: Length-only (orange - ignores risk)
start_lat, start_lon = route_coords["start"]
end_lat, end_lon = route_coords["end"]
orig_node = ox.distance.nearest_nodes(G_full, X=start_lon, Y=start_lat)
dest_node = ox.distance.nearest_nodes(G_full, X=end_lon, Y=end_lat)

length_route = nx.astar_path(
    G_full, orig_node, dest_node,
    heuristic=lambda u, v: euclidean_distance(u, v),
    weight="length"
)
length_metrics = compute_route_metrics(length_route)

# 3. ROUTE 2: Risk-aware wheelchair route (blue - avoids danger)
risk_route = nx.astar_path(
    G_full, orig_node, dest_node,
    heuristic=lambda u, v: euclidean_distance(u, v),
    weight="wheelchair_cost"
)
risk_metrics = compute_route_metrics(risk_route)

# 4. PLOT BOTH ROUTES ON SAME MAP
# Length-only route (ORANGE - may go through red zones)
# REPLACE your route plotting section with this ENHANCED version:

# 4. PLOT BOTH ROUTES (with higher weight + layer ordering)
# Length-only route (ORANGE - on TOP)
folium.PolyLine(
    locations=length_metrics['route_coords'],
    color="orange",
    weight=10,      # THICKER
    opacity=1.0,    # FULLY OPAQUE
    dash_array='10', # DASHED to distinguish
    tooltip=f"📏 LENGTH-ONLY\nAvg Risk: {length_metrics['avg_risk']:.1f}/10\nLength: {length_metrics['total_length']:.0f}m"
).add_to(m)

# Risk-aware route (BLUE - on TOP of orange) 
folium.PolyLine(
    locations=risk_metrics['route_coords'],
    color="darkblue",  # DARKER blue
    weight=12,         # THICKEST
    opacity=1.0,       # FULLY OPAQUE
    tooltip=f"🛡️ RISK-AWARE (RECOMMENDED)\nAvg Risk: {risk_metrics['avg_risk']:.1f}/10\nLength: {risk_metrics['total_length']:.0f}m"
).add_to(m)

# 5. Start/End markers (on TOP)
folium.Marker(
    [start_lat, start_lon], popup="🚩 START", 
    icon=folium.Icon(color="green", icon="play", prefix="fa", icon_size=(30,30))
).add_to(m)
folium.Marker(
    [end_lat, end_lon], popup="🏁 END", 
    icon=folium.Icon(color="red", icon="flag", prefix="fa", icon_size=(30,30))
).add_to(m)


# 6. DUAL ROUTE COMPARISON LEGEND
legend_html = f'''
<div style="position: fixed; bottom: 20px; left: 20px; width: 320px; height: 220px; 
     background-color: white; border:3px solid #333; z-index:9999; font-size:13px; 
     padding: 15px; border-radius: 10px; box-shadow: 0 4px 12px rgba(0,0,0,0.3)">
<h4 style="margin-top:0; color:#1f77b4">⚖️ ROUTE COMPARISON</h4>

<p><span style="color:orange">🟠 Length-only:</span><br>
  <small>Avg Risk: <b>{length_metrics["avg_risk"]:.1f}/10</b> | {length_metrics["total_length"]:.0f}m | 
  Danger edges: {length_metrics["high_risk_count"]}</small></p>

<p><span style="color:blue; font-weight:bold">🔵 Risk-aware:</span><br> 
  <small>Avg Risk: <b>{risk_metrics["avg_risk"]:.1f}/10</b> | {risk_metrics["total_length"]:.0f}m | 
  Danger edges: {risk_metrics["high_risk_count"]}</small></p>

<hr style="margin:10px 0">
<p><span style="color:green">🟢 Green:</span> Safe (95%)</p>
<p><span style="color:red">🔴 Red:</span> DANGER (Top 5%)</p>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# 7. Print comparison table
print("\n" + "="*70)
print("📊 ROUTE COMPARISON TABLE")
print("="*70)
print(f"{'Metric':<20} {'Length-Only':<12} {'Risk-Aware':<12} {'Winner'}")
print("-"*70)
print(f"Avg Risk Score     {length_metrics['avg_risk']:>10.2f}     {risk_metrics['avg_risk']:>10.2f}     {'🟢' if risk_metrics['avg_risk'] < length_metrics['avg_risk'] else '🔴'}")
print(f"Total Length (m)   {length_metrics['total_length']:>10.0f}     {risk_metrics['total_length']:>10.0f}     {'🟢' if length_metrics['total_length'] < risk_metrics['total_length'] else '🔴'}")
print(f"High Risk Edges    {length_metrics['high_risk_count']:>10}     {risk_metrics['high_risk_count']:>10}     {'🟢' if risk_metrics['high_risk_count'] < length_metrics['high_risk_count'] else '🔴'}")
print(f"Total Cost         {length_metrics['total_cost']:>10.1f}     {risk_metrics['total_cost']:>10.1f}     {'🟢' if risk_metrics['total_cost'] < length_metrics['total_cost'] else '🔴'}")
print("-"*70)

m

In [ ]:
# CREATE TWO SEPARATE MAPS - one for each route

# 1. Map 1: LENGTH-ONLY ROUTE (orange)
m1 = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    zoom_start=13,
    tiles="cartodbpositron"
)

# Background risk (same for both maps)
for u, v, data in list(G_full.edges(data=True))[:10000]:  # Limit for speed
    lat_u, lon_u = G_full.nodes[u]["y"], G_full.nodes[u]["x"]
    lat_v, lon_v = G_full.nodes[v]["y"], G_full.nodes[v]["x"]
    risk_norm = data.get("risk_norm", 0.0)
    color = "red" if risk_norm >= 7 else "lightgreen"
    
    folium.PolyLine(
        locations=[[lat_u, lon_u], [lat_v, lon_v]],
        color=color,
        weight=1.5 if risk_norm >= 7 else 1,
        opacity=0.3
    ).add_to(m1)

# LENGTH-ONLY ROUTE (ORANGE - thick)
folium.PolyLine(
    locations=length_metrics['route_coords'],
    color="orange",
    weight=10,
    opacity=1.0,
    tooltip=f"📏 LENGTH ROUTE\nRisk: {length_metrics['avg_risk']:.1f}/10\n{length_metrics['total_length']:.0f}m"
).add_to(m1)

# Start/End markers
folium.Marker([start_lat, start_lon], popup="START", 
              icon=folium.Icon(color="green")).add_to(m1)
folium.Marker([end_lat, end_lon], popup="END", 
              icon=folium.Icon(color="red")).add_to(m1)

# Length route legend
m1_legend = f'''
<div style="position:fixed; bottom:20px; left:20px; width:250px; height:140px; 
     background:white; border:2px solid grey; z-index:9999; padding:10px">
<b>📏 LENGTH-ONLY ROUTE</b><br>
Avg Risk: <span style="color:orange">{length_metrics['avg_risk']:.1f}/10</span><br>
Length: <span style="color:green">{length_metrics['total_length']:.0f}m</span><br>
Danger edges: {length_metrics['high_risk_count']}<br>
🟢95% Safe | 🔴Top 5% Danger
</div>
'''
m1.get_root().html.add_child(folium.Element(m1_legend))

# 2. Map 2: RISK-AWARE ROUTE (blue)
m2 = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    zoom_start=13,
    tiles="cartodbpositron"
)

# Same background risk
for u, v, data in list(G_full.edges(data=True))[:10000]:
    lat_u, lon_u = G_full.nodes[u]["y"], G_full.nodes[u]["x"]
    lat_v, lon_v = G_full.nodes[v]["y"], G_full.nodes[v]["x"]
    risk_norm = data.get("risk_norm", 0.0)
    color = "red" if risk_norm >= 7 else "lightgreen"
    
    folium.PolyLine(
        locations=[[lat_u, lon_u], [lat_v, lon_v]],
        color=color,
        weight=1.5 if risk_norm >= 7 else 1,
        opacity=0.3
    ).add_to(m2)

# RISK-AWARE ROUTE (BLUE - thickest)
folium.PolyLine(
    locations=risk_metrics['route_coords'],
    color="darkblue",
    weight=12,
    opacity=1.0,
    tooltip=f"🛡️ RISK ROUTE (RECOMMENDED)\nRisk: {risk_metrics['avg_risk']:.1f}/10\n{risk_metrics['total_length']:.0f}m"
).add_to(m2)

# Start/End markers
folium.Marker([start_lat, start_lon], popup="START", 
              icon=folium.Icon(color="green")).add_to(m2)
folium.Marker([end_lat, end_lon], popup="END", 
              icon=folium.Icon(color="red")).add_to(m2)

# Risk route legend
m2_legend = f'''
<div style="position:fixed; bottom:20px; left:20px; width:250px; height:140px; 
     background:white; border:2px solid grey; z-index:9999; padding:10px">
<b>🛡️ RISK-AWARE ROUTE</b><br>
Avg Risk: <span style="color:blue">{risk_metrics['avg_risk']:.1f}/10</span><br>
Length: <span style="color:green">{risk_metrics['total_length']:.0f}m</span><br>
Danger edges: {risk_metrics['high_risk_count']}<br>
🟢95% Safe | 🔴Top 5% Danger
</div>
'''
m2.get_root().html.add_child(folium.Element(m2_legend))

# 3. COMPARISON TABLE
print("\n" + "="*80)
print("🔄 TWO ROUTES COMPARISON")
print("="*80)
print(f"{'Metric':<18} {'LENGTH ROUTE':<15} {'RISK ROUTE':<15} {'DIFFERENCE'}")
print("-"*80)
print(f"Avg Risk         {length_metrics['avg_risk']:>12.2f}     {risk_metrics['avg_risk']:>12.2f}     {risk_metrics['avg_risk']-length_metrics['avg_risk']:>+7.2f}")
print(f"Total Length     {length_metrics['total_length']:>12.0f}m   {risk_metrics['total_length']:>12.0f}m   {risk_metrics['total_length']-length_metrics['total_length']:>+7.0f}m")
print(f"Danger Edges     {length_metrics['high_risk_count']:>12}     {risk_metrics['high_risk_count']:>12}     {'🟢 BETTER' if risk_metrics['high_risk_count'] < length_metrics['high_risk_count'] else '🔴 WORSE'}")
print("-"*80)

# Display both maps
print("\n🗺️ MAP 1: LENGTH-ONLY ROUTE (ORANGE)")
m1


In [ ]:

print("\n🗺️ MAP 2: RISK-AWARE ROUTE (BLUE - RECOMMENDED)")
m2